# TiTiler-CMR: HLS Data Demo

The Harmonized Landsat Sentinel-2 dataset is available in two collections in CMR. This example will use data from the `HLSL30.002` (Landsat) dataset.

#### Requirements
To run some of the chunks in this notebook you will need to install a few packages:

- `earthaccess`
- `folium`
- `httpx`

`!pip install folium httpx earthaccess`

In [ ]:
import earthaccess
import geojson_pydantic
import httpx
import json


from folium import GeoJson, Map, TileLayer

In [ ]:
titiler_endpoint = "http://localhost:8081"

## Identify the dataset
You can find the `HLSL30.002` dataset using the earthaccess.search_datasets function.

In [ ]:
datasets = earthaccess.search_datasets(doi="10.5067/HLS/HLSL30.002")
ds = datasets[0]

concept_id = ds["meta"]["concept-id"]
print("Concept-Id: ", concept_id)
print("Abstract: ", ds["umm"]["Abstract"])

## Examine a granule

Each granule contains the data for a single point in time for an MGRS tile.  

In [ ]:
import earthaccess
import morecantile

tms = morecantile.tms.get("WebMercatorQuad")

bounds = tms.bounds(62, 44, 7)
xmin, ymin, xmax, ymax = (round(n, 8) for n in bounds)

results = earthaccess.search_data(
    bounding_box=(xmin, ymin, xmax, ymax),
    count=1,
    concept_id=concept_id,
    temporal=("2024-02-11", "2024-02-13"),
)
print("Granules:")
print(results)
print()
print("Example of COGs URL: ")
for link in results[0].data_links(access="direct"):
    print(link)


## Demonstrate `assets_for_tile` method

While rendering `xyz` tile images, `titiler-cmr` searches for assets using the `assets_for_tile` method which converts the `xyz` tile extent into a bounding box.

In [ ]:
from titiler.cmr.backend import CMRBackend
from titiler.cmr.reader import MultiFilesBandsReader

with CMRBackend(reader=MultiFilesBandsReader) as backend:
    assets = backend.assets_for_tile(
        x=62,
        y=44,
        z=7,
        bands_regex="B[0-9][0-9]",
        concept_id=concept_id,
        temporal=("2024-02-11", "2024-02-13")
    )

print(assets[0])

## `titiler.cmr` API documentation

In [ ]:
from IPython.display import IFrame
IFrame(f"{titiler_endpoint}/api.html", 900,500)

## Display tiles in an interactive map

The `/tilejson.json` endpoint will provide a parameterized `xyz` tile URL that can be added to an interactive map.

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/WebMercatorQuad/tilejson.json",
    params = (
        ("concept_id", concept_id),
        # Datetime in form of `start_date/end_date`
        ("datetime", "2024-10-01T00:00:00Z/2024-10-10T23:59:59Z"),
        # We know that the HLS collection dataset is stored as File per Band
        # so we need to pass a `band_regex` option to assign `bands` to each URL
        ("bands_regex", "B[0-9][0-9]"),
        # titiler-cmr can work with both Zarr and COG dataset
        # but we need to tell the endpoints in advance which backend
        # to use
        ("backend", "rasterio"),
        # True Color Image B04,B03,B02
        ("bands", "B04"),
        ("bands", "B03"),
        ("bands", "B02"),
        # The data is in type of Uint16 so we need to apply some
        # rescaling/color_formula in order to create PNGs
        ("color_formula", "Gamma RGB 3.5 Saturation 1.7 Sigmoidal RGB 15 0.35"),
        # We need to set min/max zoom because we don't want to use lowerzoom level (e.g 0)
        # which will results in useless large scale query
        ("minzoom", 8),
        ("maxzoom", 13),
    )
).json()

print(r)

In [ ]:
bounds = r["bounds"]
m = Map(
    location=(47.590266824611675, -91.03729840730689),
    zoom_start=r["maxzoom"] - 2
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="NASA",
).add_to(m)
m

### Render NDVI using the `expression` parameter
The `expression` parameter can be used to render images from an expression of a combination of the individual `bands`.

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/WebMercatorQuad/tilejson.json",
    params = (
        ("concept_id", concept_id),
        # Datetime in form of `start_date/end_date`
        ("datetime", "2024-06-20T00:00:00Z/2024-06-27T23:59:59Z"),
        # We know that the HLS collection dataset is stored as File per Band
        # so we need to pass a `band_regex` option to assign `bands` to each URL
        ("bands_regex", "B[0-9][0-9]"),
        # titiler-cmr can work with both Zarr and COG dataset
        # but we need to tell the endpoints in advance which backend
        # to use
        ("backend", "rasterio"),
        # NDVI
        ("expression", "(B05-B04)/(B05+B04)"),
        # Need red (B04) and nir (B05) for NDVI
        ("bands", "B05"),
        ("bands", "B04"),
        # The data is in type of Uint16 so we need to apply some
        # rescaling/color_formula in order to create PNGs
        ("colormap_name", "viridis"),
        ("rescale", "-1,1"),
        # We need to set min/max zoom because we don't want to use lowerzoom level (e.g 0)
        # which will results in useless large scale query
        ("minzoom", 8),
        ("maxzoom", 13),
    )
).json()

m = Map(
    location=(47.9221313337365, -91.65432884883238),
    zoom_start=r["maxzoom"] - 1
)


TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="NASA",
).add_to(m)

GeoJson(geojson).add_to(m)

m

## GeoJSON Statistics
The `/statistics` endpoint can be used to get summary statistics for a geojson `Feature` or `FeatureCollection`.

In [ ]:
geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -91.65432884883238,
              47.9221313337365
            ],
            [
              -91.65432884883238,
              47.86503396133904
            ],
            [
              -91.53842043960762,
              47.86503396133904
            ],
            [
              -91.53842043960762,
              47.9221313337365
            ],
            [
              -91.65432884883238,
              47.9221313337365
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [ ]:
import json

r = httpx.post(
    f"{titiler_endpoint}/statistics",
    params=(
        ("concept_id", concept_id),
        # Datetime in form of `start_date/end_date`
        ("datetime", "2024-07-01T00:00:00Z/2024-07-10T23:59:59Z"),
        # We know that the HLS collection dataset is stored as File per Band
        # so we need to pass a `band_regex` option to assign `bands` to each URL
        ("bands_regex", "B[0-9][0-9]"),
        # titiler-cmr can work with both Zarr and COG dataset
        # but we need to tell the endpoints in advance which backend
        # to use
        ("backend", "rasterio"),
        # NDVI
        ("expression", "(B05-B04)/(B05+B04)"),
        # Need red (B04) and nir (B05) for NDVI
        ("bands", "B05"),
        ("bands", "B04"),
    ),
    json=geojson,
    timeout=30,
).json()

print(json.dumps(r, indent=2))